In [1]:
import pandas as pd
import json
import numpy
import geopy.distance
import plotly.express as px
import math
import sys
from collections import Counter

In [2]:
#dataset = pd.read_csv("partial_added_points.csv")
dataset = pd.read_csv("partial_subtracks_with_extra_points_after_dividing_only_city_centre.csv")

In [3]:
def point_distance(point1, point2):
    p1 = (point1[0], point1[1])
    p2 = (point2[0], point2[1])
    dist = geopy.distance.distance(p1, p2).m
    if dist < 30:
        return 0
    else:
        return 1

In [4]:
def actual_distance(point1, point2):
    p1 = (point1[0], point1[1])
    p2 = (point2[0], point2[1])
    dist = geopy.distance.distance(p1, p2).m
    return round(dist)

In [18]:
numpy.set_printoptions(threshold=sys.maxsize)


def create_matrix(track1, track2):
    matrix = numpy.matrix(numpy.ones((len(track1) + 1, len(track2) + 1))*numpy.inf)
    matrix[0, 0] = 0

    #for r in range(1, len(track1) + 1):
    #    matrix[r, 0] = numpy.inf

    #for c in range(1, len(track2) + 1):
    #    matrix[0, c] = numpy.inf
    for i in range(1, len(track1) + 1):
        j=1
        while j < len(track2) + 1:
        #for j in range(1, len(track2) + 1):
            dist = actual_distance(track1[i - 1], track2[j - 1])
            matrix[i, j] = (
                min(matrix[i - 1, j], min(matrix[i, j - 1], matrix[i - 1, j - 1]))
                + dist
            )
            #if matrix[i, j]>
            #if j> 2 and dist>500:
            #    j = j + math.floor(dist/40)
            j+=1

    return matrix


track1 = dataset.loc[dataset["id"] == "T1"].values.tolist()
track2 = dataset.loc[dataset["id"] == "T3"].values.tolist()
matrix = create_matrix(track1, track2)
print(matrix)

[[   0.   inf   inf   inf   inf   inf   inf   inf   inf   inf   inf   inf
    inf   inf   inf   inf   inf   inf   inf   inf   inf   inf]
 [  inf    7.   41.   75.  125.  180.  244.  319.  394.  447.  484.  521.
   570.  619.  684.  768.  852.  955. 1078. 1220. 1382. 1564.]
 [  inf   16.   28.   49.   86.  128.  180.  244.  308.  353.  388.  423.
   477.  531.  604.  697.  790.  902. 1033. 1184. 1354. 1543.]
 [  inf   49.   21.   26.   39.   63.  101.  155.  209.  253.  301.  349.
   423.  497.  591.  706.  812.  924. 1055. 1204. 1375. 1564.]
 [  inf   82.   26.   26.   39.   63.  101.  155.  209.  253.  301.  349.
   423.  497.  591.  706.  821.  946. 1077. 1226. 1395. 1585.]
 [  inf  123.   37.   37.   31.   48.   80.  128.  176.  218.  270.  322.
   401.  480.  580.  701.  822.  960. 1104. 1253. 1421. 1609.]
 [  inf  172.   63.   63.   53.   38.   49.   75.  101.  128.  173.  218.
   292.  366.  462.  581.  700.  835.  987. 1157. 1345. 1552.]
 [  inf  221.   89.   89.   75.   45.   4

In [6]:
def backtrack(matrix, track1, track2):
    a = []
    i = len(track1)
    j = len(track2)

    while i > 0 and j > 0:

        minimum = min(matrix[i - 1, j - 1], min(matrix[i - 1, j], matrix[i, j - 1]))
        if matrix[i - 1, j] == minimum:
            i = i - 1
        elif matrix[i, j - 1] == minimum:
            j = j - 1
        elif matrix[i - 1, j - 1] == minimum:
            j = j - 1
            i = i - 1
        else:
            print("hallais")
        if (
            point_distance(
                (track1[i - 1][0], track1[i - 1][1]),
                (track2[j - 1][0], track2[j - 1][1]),
            )
            == 0
        ):
            point = [track1[i - 1][0], track1[i - 1][1], track2[0][2]]
            if len(a) == 0 or a[0] != point:
                a.insert(0, point)
    return a


align = backtrack(matrix, track1, track2)
print(align)

[[41.148522, -8.585676, 'T3'], [41.148639, -8.585712000000001, 'T3'], [41.148855000000005, -8.585685000000002, 'T3'], [41.14892699999999, -8.585730000000002, 'T3'], [41.148963, -8.585982, 'T3'], [41.1489585, -8.586189000000001, 'T3'], [41.148954, -8.586396, 'T3'], [41.148837, -8.586234000000001, 'T3'], [41.14872, -8.586072000000001, 'T3'], [41.14872, -8.586072, 'T3'], [41.1485454, -8.5861224, 'T3'], [41.1483708, -8.586172800000002, 'T3'], [41.1481962, -8.586223200000003, 'T3'], [41.14802159999999, -8.586273600000004, 'T3'], [41.14784699999999, -8.586324000000005, 'T3'], [41.147847, -8.586324000000001, 'T3'], [41.14775025, -8.586492750000001, 'T3'], [41.1476535, -8.586661500000002, 'T3'], [41.14755675000001, -8.586830250000002, 'T3'], [41.14746000000001, -8.586999000000002, 'T3'], [41.14746, -8.586999, 'T3'], [41.147307, -8.5867875, 'T3']]


In [7]:
track_ids = pd.read_csv("partial_subtracks_ids_add_points_after_dividing.csv").values.tolist()


In [19]:
def Dynamic_Time_Warping(dataset, trackid, nr):
    track = dataset.loc[dataset["id"] == trackid]
    track = pd.DataFrame.drop_duplicates(track)
    print("track1: ", trackid)
    alignments = []
    for j in range(nr + 1, len(track_ids)):
        next_track = dataset.loc[dataset["id"] == track_ids[j][0]]
        matrix = create_matrix(track.values.tolist(), next_track.values.tolist())
        alignment = backtrack(matrix, track.values.tolist(), next_track.values.tolist())
        if len(alignment) > 3:
            alignments.extend(alignment)
    return alignments


align = Dynamic_Time_Warping(dataset, "T2", 1)
print(align)

track1:  T2
[[41.14557, -8.610876000000001, 'T32'], [41.145579000000005, -8.610858, 'T32'], [41.145768, -8.610903, 'T32'], [41.145909, -8.61075, 'T32'], [41.14605, -8.610596999999999, 'T32'], [41.146191, -8.610443999999998, 'T32'], [41.146191, -8.610444, 'T32'], [41.1462855, -8.610277499999999, 'T32'], [41.14638, -8.610111, 'T32'], [41.1464745, -8.609944500000001, 'T32'], [41.14656900000001, -8.609778000000002, 'T32'], [41.14666350000001, -8.609611500000003, 'T32'], [41.14675800000001, -8.609445000000004, 'T32'], [41.146758, -8.609445000000001, 'T32'], [41.146878, -8.609262000000001, 'T32'], [41.146998, -8.609079000000001, 'T32'], [41.147118000000006, -8.608896000000001, 'T32'], [41.147118, -8.608896, 'T32'], [41.147127, -8.608968, 'T32'], [41.1473295, -8.6088375, 'T32'], [41.149962, -8.607357, 'T33'], [41.1501315, -8.607267, 'T33'], [41.150301, -8.607177, 'T33'], [41.1504705, -8.607087, 'T33'], [41.15064, -8.606997, 'T33'], [41.1508095, -8.606907, 'T33'], [41.15097899999999, -8.606817

In [9]:
def create_list_of_points(tracknr, alignments):

    track = dataset.loc[dataset["id"] == tracknr]
    track = pd.DataFrame.drop_duplicates(track)
    points = pd.DataFrame(columns=["lat", "lon", "ids"])
    align = pd.DataFrame(alignments, columns=["lat", "lon", "id"])

    for i in range(len(track)):
        lat = align.loc[align["lat"] == track.iloc[i]["lat"]]
        points.loc[i] = [
            track.iloc[i]["lat"],
            track.iloc[i]["lon"],
            lat["id"].values.tolist(),
        ]
    return points

In [10]:
def remove_infrequent_points(points):
    point_list = pd.DataFrame(points, columns=["lat", "lon", "ids"])
    for i in range(len(point_list)):
        val = len(point_list.loc[i]["ids"])
        point_list.at[i, "ids"] = val
    return point_list

In [11]:
def gather_sequences(points):
    i = 0
    final_records = pd.DataFrame(columns=["polyline", "value"])
    for group in range(points.iloc[0]["group"], -1, -1):
        records = pd.DataFrame(columns=["polyline", "value"])
        sequence = []
        value = 0
        while i < len(points) and points.iloc[i]["group"] == group:
            if value == points.iloc[i]["ids"] or value == 0:
                sequence.append(
                    [
                        points.index[i],
                        points.iloc[i]["lat"],
                        points.iloc[i]["lon"],
                    ]
                )
                value = points.iloc[i]["ids"]
                i += 1
            else:
                sequence.sort()
                new_sequence = sequence.copy()
                records.loc[len(records)] = [
                    new_sequence,
                    (len(new_sequence) * 1.0) + value,
                ]
                value = points.iloc[i]["ids"]

        sequence.sort()
        new_sequence = sequence.copy()
        records.loc[len(records)] = [
            new_sequence,
            (len(new_sequence) * 1.0) + value,
        ]
        if len(records) < 2:
            final_records = pd.concat([final_records, records])
        else:
            highest_value = records["value"].idxmax()
            final_records = final_records.append(records.loc[highest_value])

    return final_records

In [20]:
def hele_sullamitten(dataset):
    most_tracked_roads = pd.DataFrame(columns=["polyline", "value"])
    for i in range(len(track_ids)):
        track = track_ids[i][0]
        alignment = Dynamic_Time_Warping(dataset, track, i)
        points = create_list_of_points(track, alignment)
        new_points = remove_infrequent_points(points)
        new_points = new_points[new_points.ids > 8]
        if not new_points.empty:
            new_points["group"] = (new_points.index.to_series().diff() > 1).cumsum()
            new_points.sort_values(by=["group", "ids"], ascending=False, inplace=True)
            records = gather_sequences(new_points)
            most_tracked_roads = pd.concat([most_tracked_roads, records])
            print(most_tracked_roads)
    return most_tracked_roads


res = hele_sullamitten(dataset)

track1:  T1
                                            polyline  value
2  [[1, 41.148639, -8.585712000000001], [2, 41.14...   27.0
track1:  T2
                                            polyline  value
2  [[1, 41.148639, -8.585712000000001], [2, 41.14...   27.0
0  [[35, 41.149899000000005, -8.607348], [36, 41....   24.0
5  [[0, 41.14557, -8.610876000000001], [1, 41.145...   30.0
track1:  T2334
                                            polyline  value
2  [[1, 41.148639, -8.585712000000001], [2, 41.14...   27.0
0  [[35, 41.149899000000005, -8.607348], [36, 41....   24.0
5  [[0, 41.14557, -8.610876000000001], [1, 41.145...   30.0
0  [[9, 41.159943000000005, -8.604513000000006], ...   12.0
track1:  T3
                                            polyline  value
2  [[1, 41.148639, -8.585712000000001], [2, 41.14...   27.0
0  [[35, 41.149899000000005, -8.607348], [36, 41....   24.0
5  [[0, 41.14557, -8.610876000000001], [1, 41.145...   30.0
0  [[9, 41.159943000000005, -8.604513000000006], 

track1:  T15
                                            polyline  value
2  [[1, 41.148639, -8.585712000000001], [2, 41.14...   27.0
0  [[35, 41.149899000000005, -8.607348], [36, 41....   24.0
5  [[0, 41.14557, -8.610876000000001], [1, 41.145...   30.0
0  [[9, 41.159943000000005, -8.604513000000006], ...   12.0
0  [[1, 41.148828, -8.585730000000002], [2, 41.14...   25.0
0  [[50, 41.160816, -8.581499999999993], [51, 41....   14.0
0  [[22, 41.156487, -8.582985000000004], [23, 41....   14.0
0  [[4, 41.153832, -8.582021999999998], [5, 41.15...   15.0
0  [[6, 41.140926, -8.615088000000002], [7, 41.14...   12.0
0  [[1, 41.1489, -8.585640000000001], [2, 41.1489...   22.0
0  [[3, 41.152923, -8.581185000000001], [4, 41.15...   14.0
0  [[1, 41.148603, -8.585703], [2, 41.148603, -8....   51.0
0  [[60, 41.160321, -8.581725000000004], [61, 41....   12.0
0  [[36, 41.15640600000001, -8.582985000000003], ...   12.0
0  [[69, 41.168277, -8.587637999999998], [70, 41....   15.0
0  [[28, 41.165118, -8.5889

                                             polyline  value
2   [[1, 41.148639, -8.585712000000001], [2, 41.14...   27.0
0   [[35, 41.149899000000005, -8.607348], [36, 41....   24.0
5   [[0, 41.14557, -8.610876000000001], [1, 41.145...   30.0
0   [[9, 41.159943000000005, -8.604513000000006], ...   12.0
0   [[1, 41.148828, -8.585730000000002], [2, 41.14...   25.0
0   [[50, 41.160816, -8.581499999999993], [51, 41....   14.0
0   [[22, 41.156487, -8.582985000000004], [23, 41....   14.0
0   [[4, 41.153832, -8.582021999999998], [5, 41.15...   15.0
0   [[6, 41.140926, -8.615088000000002], [7, 41.14...   12.0
0   [[1, 41.1489, -8.585640000000001], [2, 41.1489...   22.0
0   [[3, 41.152923, -8.581185000000001], [4, 41.15...   14.0
0   [[1, 41.148603, -8.585703], [2, 41.148603, -8....   51.0
0   [[60, 41.160321, -8.581725000000004], [61, 41....   12.0
0   [[36, 41.15640600000001, -8.582985000000003], ...   12.0
0   [[69, 41.168277, -8.587637999999998], [70, 41....   15.0
0   [[28, 41.165118, -8.

track1:  T23
                                             polyline  value
2   [[1, 41.148639, -8.585712000000001], [2, 41.14...   27.0
0   [[35, 41.149899000000005, -8.607348], [36, 41....   24.0
5   [[0, 41.14557, -8.610876000000001], [1, 41.145...   30.0
0   [[9, 41.159943000000005, -8.604513000000006], ...   12.0
0   [[1, 41.148828, -8.585730000000002], [2, 41.14...   25.0
0   [[50, 41.160816, -8.581499999999993], [51, 41....   14.0
0   [[22, 41.156487, -8.582985000000004], [23, 41....   14.0
0   [[4, 41.153832, -8.582021999999998], [5, 41.15...   15.0
0   [[6, 41.140926, -8.615088000000002], [7, 41.14...   12.0
0   [[1, 41.1489, -8.585640000000001], [2, 41.1489...   22.0
0   [[3, 41.152923, -8.581185000000001], [4, 41.15...   14.0
0   [[1, 41.148603, -8.585703], [2, 41.148603, -8....   51.0
0   [[60, 41.160321, -8.581725000000004], [61, 41....   12.0
0   [[36, 41.15640600000001, -8.582985000000003], ...   12.0
0   [[69, 41.168277, -8.587637999999998], [70, 41....   15.0
0   [[28, 4

                                             polyline  value
2   [[1, 41.148639, -8.585712000000001], [2, 41.14...   27.0
0   [[35, 41.149899000000005, -8.607348], [36, 41....   24.0
5   [[0, 41.14557, -8.610876000000001], [1, 41.145...   30.0
0   [[9, 41.159943000000005, -8.604513000000006], ...   12.0
0   [[1, 41.148828, -8.585730000000002], [2, 41.14...   25.0
..                                                ...    ...
3   [[92, 41.14632109090909, -8.617954090909091], ...   16.0
0   [[76, 41.146542, -8.620379999999997], [77, 41....   14.0
0   [[59, 41.147784, -8.622629999999997], [60, 41....   13.0
0   [[38, 41.14944, -8.625617999999996], [39, 41.1...   14.0
0   [[21, 41.152392, -8.626157999999998], [22, 41....   13.0

[64 rows x 2 columns]
track1:  T30
                                             polyline  value
2   [[1, 41.148639, -8.585712000000001], [2, 41.14...   27.0
0   [[35, 41.149899000000005, -8.607348], [36, 41....   24.0
5   [[0, 41.14557, -8.610876000000001], [1, 41.14

...

                                             polyline  value
2   [[1, 41.148639, -8.585712000000001], [2, 41.14...   27.0
0   [[35, 41.149899000000005, -8.607348], [36, 41....   24.0
5   [[0, 41.14557, -8.610876000000001], [1, 41.145...   30.0
0   [[9, 41.159943000000005, -8.604513000000006], ...   12.0
0   [[1, 41.148828, -8.585730000000002], [2, 41.14...   25.0
..                                                ...    ...
2   [[27, 41.147370000000016, -8.608554000000005],...   26.0
0   [[13, 41.146047, -8.607068999999997], [14, 41....   12.0
0   [[6, 41.144904, -8.607051000000002], [7, 41.14...   14.0
0   [[1, 41.144589, -8.606439], [2, 41.144589, -8....   12.0
0   [[9, 41.154300000000006, -8.64828], [10, 41.15...   16.0

[243 rows x 2 columns]
track1:  T186
track1:  T186364640
track1:  T1861094650
track1:  T186264668
track1:  T188
track1:  T188644708
track1:  T188814721
track1:  T18884729
track1:  T189
track1:  T18914737
track1:  T190
track1:  T190474755
track1:  T190784760
track1:  

track1:  T217265741
track1:  T218
                                             polyline  value
2   [[1, 41.148639, -8.585712000000001], [2, 41.14...   27.0
0   [[35, 41.149899000000005, -8.607348], [36, 41....   24.0
5   [[0, 41.14557, -8.610876000000001], [1, 41.145...   30.0
0   [[9, 41.159943000000005, -8.604513000000006], ...   12.0
0   [[1, 41.148828, -8.585730000000002], [2, 41.14...   25.0
..                                                ...    ...
0   [[82, 41.147817524999866, -8.619900075000029],...   14.0
1   [[71, 41.1478892999999, -8.61788790000002], [7...   16.0
1   [[49, 41.14803284999998, -8.613863550000005], ...   14.0
1   [[38, 41.14794342857143, -8.612010000000001], ...   19.0
0   [[27, 41.147649, -8.610642000000004], [28, 41....   18.0

[290 rows x 2 columns]
track1:  T218825795
track1:  T218235806
                                             polyline  value
2   [[1, 41.148639, -8.585712000000001], [2, 41.14...   27.0
0   [[35, 41.149899000000005, -8.607348], [36, 4

track1:  T240
                                             polyline  value
2   [[1, 41.148639, -8.585712000000001], [2, 41.14...   27.0
0   [[35, 41.149899000000005, -8.607348], [36, 41....   24.0
5   [[0, 41.14557, -8.610876000000001], [1, 41.145...   30.0
0   [[9, 41.159943000000005, -8.604513000000006], ...   12.0
0   [[1, 41.148828, -8.585730000000002], [2, 41.14...   25.0
..                                                ...    ...
1   [[16, 41.147928, -8.610930000000002], [17, 41....   31.0
0   [[1, 41.14602, -8.612505], [2, 41.14603125, -8...   16.0
0               [[34, 41.147739, -8.618012999999998]]   16.0
0   [[27, 41.147235, -8.616978000000005], [28, 41....   14.0
0       [[11, 41.14789739999999, -8.614486799999998]]   12.0

[320 rows x 2 columns]
track1:  T240236908
                                             polyline  value
2   [[1, 41.148639, -8.585712000000001], [2, 41.14...   27.0
0   [[35, 41.149899000000005, -8.607348], [36, 41....   24.0
5   [[0, 41.14557, -8.61087

                                             polyline  value
2   [[1, 41.148639, -8.585712000000001], [2, 41.14...   27.0
0   [[35, 41.149899000000005, -8.607348], [36, 41....   24.0
5   [[0, 41.14557, -8.610876000000001], [1, 41.145...   30.0
0   [[9, 41.159943000000005, -8.604513000000006], ...   12.0
0   [[1, 41.148828, -8.585730000000002], [2, 41.14...   25.0
..                                                ...    ...
0      [[19, 41.145435000000006, -8.607564000000002]]   13.0
1   [[159, 41.14889742857142, -8.625209142857145],...   33.0
2   [[127, 41.14756566666664, -8.620134000000002],...   37.0
0   [[6, 41.145993, -8.612406000000002], [7, 41.14...   12.0
0   [[0, 41.145957, -8.61282], [1, 41.145948, -8.6...   16.0

[347 rows x 2 columns]
track1:  T258
track1:  T259
track1:  T260677397
track1:  T260487407
track1:  T260497412
track1:  T260287420
track1:  T261
                                             polyline  value
2   [[1, 41.148639, -8.585712000000001], [2, 41.14...   27.0


track1:  T28317852
track1:  T284
                                             polyline  value
2   [[1, 41.148639, -8.585712000000001], [2, 41.14...   27.0
0   [[35, 41.149899000000005, -8.607348], [36, 41....   24.0
5   [[0, 41.14557, -8.610876000000001], [1, 41.145...   30.0
0   [[9, 41.159943000000005, -8.604513000000006], ...   12.0
0   [[1, 41.148828, -8.585730000000002], [2, 41.14...   25.0
..                                                ...    ...
0   [[72, 41.148603, -8.625068999999998], [73, 41....   12.0
0   [[24, 41.147865, -8.610642], [25, 41.147865, -...   12.0
0                        [[12, 41.145948, -8.612397]]   11.0
0                         [[10, 41.145948, -8.61246]]   11.0
0   [[26, 41.14908, -8.610588000000003], [27, 41.1...   14.0

[369 rows x 2 columns]
track1:  T286
track1:  T288
track1:  T290
track1:  T290677909
track1:  T291
                                             polyline  value
2   [[1, 41.148639, -8.585712000000001], [2, 41.14...   27.0
0   [[35, 41.

In [21]:
pd.DataFrame.to_csv(res, "./DTW_result_final.csv", index=True)